<a href="https://colab.research.google.com/github/Fetis789/hackatonvk/blob/main/Vkhackaton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 124.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 147.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.6 MB/s eta 0:00:00


In [2]:
import os

import torch
import torch.nn.functional as F
import lightgbm

import matplotlib.pyplot as plt
import seaborn as sns
import transformers
import pandas as pd
import numpy as np
import random
import ipywidgets
import re

from tqdm.notebook import trange, tqdm
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizer, BertTokenizer
from transformers import BertForSequenceClassification
from sklearn.model_selection import GroupShuffleSplit
from nltk.stem import *
from nltk.corpus import stopwords
import nltk

#from dataset import *
#from model import *
#from trainer import Trainer

In [3]:
test = pd.read_json("ranking_test.jsonl", lines = True)
train = pd.read_json("ranking_train.jsonl", lines = True)

ValueError: ignored

In [ ]:
test.head()

,text,comments
0,"iOS 8.0.1 released, broken on iPhone 6 models,...",[{'text': 'I&#x27;m still waiting for them to ...
1,Ask HN: How do US HNers get their health insur...,[{'text': 'Get it from your employer. It&#x27;...
2,San Diego Researcher Crowdfunding Patent-Free ...,[{'text': 'What I don&#x27;t understand is why...
3,Rethinking the origins of the universe,[{'text': 'I&#x27;m not a physicist. I imagin...
4,SlackTextViewController: A new growing text in...,[{'text': 'As someone that doesn&#x27;t do iOS...


In [ ]:
train.head()

,text,comments
0,How many summer Y Combinator fundees decided n...,[{'text': 'Going back to school is not identic...
1,CBS acquires last.fm for $280m,[{'text': 'It will be curious to see where thi...
2,How Costco Became the Anti-Wal-Mart,[{'text': 'I really hate it when people falsel...
3,"Fortune Favors Big Turds | Screw The Money, Th...",[{'text': 'His real point is that something ca...
4,StartupWeekend: 70 Founders Create One Company...,[{'text': 'Looks like someone hasn't read The ...


In [ ]:
train["comments"][1][3]

{'text': "I don't understand what they do that is worth $70M a year. ",
 'score': 3}

In [ ]:
len(train)

88107

In [ ]:
def data_to_right_format(data):
  data_rank = pd.DataFrame(columns = ["post", "comment", "rank"])
  j = 0
  while j <= len(data):
    cur_list = train["comments"][j]
    for i in range(5):
      new_row = {}
      new_row["post"] = train["text"][j]
      new_row["comment"] = cur_list[i]["text"]
      new_row["rank"] = cur_list[i]["score"]
      data_rank = data_rank.append(new_row, ignore_index = True)
    j+=1
  return data_rank


def data_to_right_format_fast(data):
  data_rank = pd.DataFrame(columns = ["post", "comment", "rank"])
  j = 0
  posts = []
  comments = []
  ranks = []
  while j < len(data):
    cur_list = train["comments"][j]
    for i in range(5):
      new_row = {}
      posts.append(train["text"][j])
      comments.append(cur_list[i]["text"])
      ranks.append(cur_list[i]["score"])
    j+=1
  data_rank["post"] = posts
  data_rank["comment"] = comments
  data_rank["rank"] = ranks
  return data_rank


In [ ]:
train_rank = data_to_right_format_fast(train)
train_rank.head(10)

,post,comment,rank
0,How many summer Y Combinator fundees decided n...,Going back to school is not identical with giv...,0
1,How many summer Y Combinator fundees decided n...,There will invariably be those who don't see t...,1
2,How many summer Y Combinator fundees decided n...,For me school is a way to be connected to what...,2
3,How many summer Y Combinator fundees decided n...,I guess it really depends on how hungry you ar...,3
4,How many summer Y Combinator fundees decided n...,I know pollground decided to go back to school...,4
5,CBS acquires last.fm for $280m,It will be curious to see where this heads in ...,0
6,CBS acquires last.fm for $280m,Does this mean that there's now a big-name com...,1
7,CBS acquires last.fm for $280m,Also on BBC News: http://news.bbc.co.uk/1/low...,2
8,CBS acquires last.fm for $280m,I don't understand what they do that is worth ...,3
9,CBS acquires last.fm for $280m,sold out too cheaply. given their leadership p...,4


In [ ]:
need_cols = ["post", "comment"]
target_col = ["rank"]

X = train_rank[need_cols]
y = train_rank[target_col]

In [ ]:
splitter = GroupShuffleSplit(test_size=0.25, n_splits=1, random_state = 7)
split = splitter.split(X, y, groups = X["post"])
train_inds, test_inds = next(split)


X_beg_train = X.iloc[train_inds]
y_train = y.iloc[train_inds]
X_beg_test = X.iloc[test_inds]
y_test = y.iloc[test_inds]

In [ ]:
qids_train = X_beg_train.groupby("post")["post"].count().to_numpy()
X_train = X_beg_train["comment"]

qids_test = X_beg_test.groupby("post")["post"].count().to_numpy()
X_test = X_beg_test["comment"]



In [ ]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
)

In [ ]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_test, y_test)],
    eval_group=[qids_test],
    eval_at=10,
    verbose=10,
)

ValueError: ignored